In [2]:
import csv
import re
import pandas as pd
import collections
import numpy as np
from scipy import stats
from math import trunc
import threading
import multiprocessing as mp
from Bio import SeqIO
import pysam
import scipy
import math
import os
import subprocess
import shutil
from collections import Counter
from difflib import SequenceMatcher
from itertools import combinations
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.stats import fisher_exact
import matplotlib.pyplot as plt
import seaborn as sns

# UMIC-seq

## 1. Extraction of Sample Barcodes and split the raw data

In [ ]:
input_path = '' # The path of fasta files output by UMIC-seq_helper.py
output_path = '' # The path to output the splited data
raw_data_path = '' # The path of raw data

Demultiplex_files = [f for f in os.listdir(input_path) if f.endswith('.fasta')] 

BC_reads_id = []
BC_reads_sequences = []
BC_id = []

for Demultiplex_file in Demultiplex_files:
    SampleBC = Demultiplex_file.split('_')[-1].split('.')[0]
    BC_fasta_file = (input_path + Demultiplex_file)
    for record in SeqIO.parse(BC_fasta_file, "fasta"):
        BC_reads_id.append(record.id)
        BC_reads_sequences.append(str(record.seq))
        BC_id.append(SampleBC)

BC_reads_df = pd.DataFrame({'Reads_ID': BC_reads_id, 'Sequence': BC_reads_sequences, 'BC_ID': BC_id})
print('Finish: Sample Barcode reads to dataframe')

BC_reads_df['info'] = BC_reads_df['Reads_ID'] + '_' + BC_reads_df['BC_ID']
BC_reads_df = BC_reads_df.drop_duplicates('info').reset_index(drop = True)

# raw data to dataframe

data = []
with open(raw_data_path, 'r') as file:
    lines = file.readlines()
    buffer = []
    for line in lines:
        buffer.append(line.strip())
        if len(buffer) == 4:
            try:
                read_id = buffer[0]
                seq = buffer[1]
                plus = buffer[2]
                qual = buffer[3]
                data.append([read_id, seq, plus, qual])
            except Exception as e:
                print(f"Skipping invalid entry: {e}")
            buffer = []

clean_fq = pd.DataFrame(data, columns=['read_id', 'seq', '+', 'qual'])
clean_fq['read_id_split'] = clean_fq['read_id'].str.split(' ').str.get(0).str[1:]
clean_fq = clean_fq.drop_duplicates('read_id_split').reset_index(drop = True)

print('Finish: fastq to dataframe')

# split the raw data by sample BC

for BC_ID, group in BC_reads_df.groupby("BC_ID"):
    Reads_ID_list = group["Reads_ID"].tolist()
    output_file = f"{BC_ID}.fq"
    print(BC_ID)
    clean_fq_subset = clean_fq[clean_fq['read_id_split'].isin(Reads_ID_list)].reset_index(drop = True)

    with open((output_path + output_file), "w") as output_fq:
        for i in range(len(clean_fq_subset['read_id_split'])):
            output_fq.write(clean_fq_subset['read_id'][i] + '\n')
            output_fq.write(clean_fq_subset['seq'][i] + '\n')
            output_fq.write(clean_fq_subset['+'][i] + '\n')
            output_fq.write(clean_fq_subset['qual'][i] + '\n')

## 2. Extraction of UMIs for each BC_xx.fq

In [3]:
# Generate commands for extracting UMIs from each BC_xx.fq file

input_folder = '' # The path of the file containing BC_xx.fq
output_folder = '' # The path of output data
probe_path = '' # The path of the fasta file containing the probe sequence

input_files = [f for f in os.listdir(input_folder) if f.endswith('.fq')]

command_list = []

for input_file in input_files:
    input_file_path = os.path.join(input_folder, input_file)
    output_file = 'ExtractedUMIs_' + input_file.replace(".fq", ".fasta")
    output_file_path = os.path.join(output_folder, output_file)

    mpileup_command = f"'python UMIC-seq.py UMIextract --input {input_file_path} --probe {probe_path} --umi_loc down --umi_len 40 --output {output_file_path}'"
    command_list.append(mpileup_command)

with open((output_folder + 'ExtractedUMIs_command_list.txt'), "w") as file:
    for mpileup_command in command_list:
        file.write(mpileup_command+'\n')

## 3. Clustering approximation (Cluster test)

In [5]:
# Generate commands for clustering approximation for each ExtractedUMIs.fasta file

input_folder = '' # The path of the file containing extracted UMIs
output_folder = '' # The path of output data

input_files = [f for f in os.listdir(input_folder) if f.endswith('.fasta')]

command_list = []

for input_file in input_files:
    input_file_path = os.path.join(input_folder, input_file)
    output_file = 'UMIclustertest_' + input_file.split('.')[0].split('_')[1]
    output_file_path = os.path.join(output_folder, output_file)

    mpileup_command = f"'python UMIC-seq.py clustertest --input {input_file_path} --steps 20 60 5 --output {output_file_path}'"
    command_list.append(mpileup_command)

with open((output_folder + 'UMIclustertest_command_list.txt'), "w") as file:
    for mpileup_command in command_list:
        file.write(mpileup_command+'\n')

## 4. Full clustering

In [4]:
### Generate commands for Clusterfull for each ExtractedUMIs.fasta file

input_folder_1 = '' # The path of the file containing extracted UMIs
input_folder_2 = '' # The path of the file containing BC_xx.fq
output_folder = '' # The path of output data

input_files_name = [f.split('_')[1].split('.')[0] for f in os.listdir(input_folder_1) if f.endswith('.fasta')]
input_files_name.sort()
command_list = []

for input_file in input_files_name:
    input_file_path_1 = input_folder_1 + 'ExtractedUMIs_' + input_file + '.fasta'
    input_file_path_2 = input_folder_2 + input_file + '.fq'

    output_file = 'UMIclusterfull_' + input_file
    output_file_path = output_folder + output_file

    mpileup_command = f"'python UMIC-seq.py clusterfull --input {input_file_path_1} --reads {input_file_path_2} --aln_thresh 50 --size_thresh 10 --output {output_file_path} --stop_thresh 0'"
    command_list.append(mpileup_command)

with open((output_folder + 'UMIclusterfull_command_list.txt'), "w") as file:
    for mpileup_command in command_list:
        file.write(mpileup_command+'\n')

## 5. Generation of consensus sequence

### a. Merge all reads form all sample

In [8]:
input_path = '' # The input path of the file containing Clusterfull files

input_files_name = [f.split('_')[1] for f in os.listdir(input_path) if f.endswith('.pdf')]

reads_id = []
reads_sequences = []

for file_name in input_files_name:
    cluster_files_name = [n.split('.')[0].split('_')[1] for n in os.listdir(input_path + 'UMIclusterfull_' + file_name) if n.endswith('.fasta')]
    print(file_name)
    for cluster in cluster_files_name:
        print(cluster)
        fasta_file = (input_path + 'UMIclusterfull_' + file_name + '/cluster_' + cluster + '.fasta')
        for record in SeqIO.parse(fasta_file, "fasta"):
            reads_id.append((record.id + '-' + file_name + '-' + cluster))
            reads_sequences.append(str(record.seq))

reads_df = pd.DataFrame({'Reads_ID': reads_id, 'Sequence': reads_sequences})

with open((input_path + 'BC_UMI_merge.fasta'), 'w') as fasta_file_merge:
    for index, row in reads_df.iterrows():
        reads_id = row['Reads_ID']
        seq = row['Sequence']
        fasta_file_merge.write(f'>{reads_id}\n{seq}\n')


BC06
1
2
BC13
1
2
3
BC30
1
2
3
BC02
1
2
3
4
5
6
7
8
9
BC25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
BC04
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


### b. BC_UMI_merge.fasta.bam generated and split the reads by sample and cluster

In [4]:
# Mapping reads to reference sequence and sort the bam file
# nohup minimap2 -ax map-ont -t 50 reference.fa BC_UMI_merge.fasta -o BC_UMI_merge.fasta.bam &
# nohup samtools sort BC_UMI_merge.fasta.bam -o BC_UMI_merge.fasta.sorted.bam &

path = '' # The path to output the split bam file

bam_file = pysam.AlignmentFile(('BC_UMI_merge.fasta.sorted.bam'), 'rb')
header = str(bam_file.header)

SampleBC_Cluster_dict = {}

for read in bam_file:
    BC_cluster = '_'.join(str(read.query_name).split('-')[-2:])
    if BC_cluster in SampleBC_Cluster_dict:
        SampleBC_Cluster_dict[BC_cluster].append(read)
    else:
        SampleBC_Cluster_dict[BC_cluster] = [read]

SampleBC_Cluster_list = []
for SampleBC_Cluster, reads in SampleBC_Cluster_dict.items():
    SampleBC_Cluster_list.append(SampleBC_Cluster)
    print(SampleBC_Cluster)
    with open((path + 'BAM_and_mpileup_split/' + f"{SampleBC_Cluster}.bam"), "w") as file:
        file.write(header)
        for read in reads:
            file.write(f"{read.tostring()}\n")


[E::idx_find_and_load] Could not retrieve index file for '/Dell/Dell15/zhanggy/project/7-GHT-UMIC-seq/tmp/Consensus_seq/BC_UMI_merge.fasta.sorted.bam'


BC06_1
BC06_2
BC13_1
BC13_2
BC13_3
BC30_1
BC30_2
BC30_3
BC02_1
BC02_2
BC02_3
BC02_4
BC02_5
BC02_6
BC02_7
BC02_8
BC02_9
BC25_1
BC25_2
BC25_3
BC25_4
BC25_5
BC25_6
BC25_7
BC25_8
BC25_9
BC25_10
BC25_11
BC25_12
BC25_13
BC25_14
BC25_15
BC25_16
BC25_17
BC25_18
BC25_19
BC25_20
BC25_21
BC25_22
BC25_23
BC25_24
BC25_25
BC25_26
BC25_27
BC25_28
BC25_29
BC25_30
BC25_31
BC25_33
BC25_34
BC25_35
BC25_36
BC25_37
BC25_38
BC25_39
BC25_40
BC25_41
BC25_42
BC25_43
BC25_44
BC25_45
BC25_46
BC25_47
BC25_48
BC25_49
BC25_50
BC25_51
BC25_52
BC25_53
BC25_54
BC25_55
BC25_56
BC25_57
BC25_58
BC25_59
BC25_60
BC25_61
BC25_62
BC25_63
BC25_64
BC25_65
BC25_66
BC25_67
BC25_68
BC25_69
BC25_70
BC25_71
BC25_72
BC25_73
BC25_74
BC25_75
BC25_76
BC25_77
BC25_78
BC25_79
BC25_80
BC25_81
BC25_82
BC25_83
BC25_84
BC25_85
BC25_86
BC25_87
BC25_88
BC25_89
BC25_90
BC25_91
BC25_92
BC25_93
BC25_94
BC25_95
BC25_96
BC25_97
BC25_99
BC25_100
BC25_101
BC25_103
BC25_104
BC25_105
BC25_106
BC25_107
BC25_108
BC25_109
BC25_110
BC25_111
BC25_112
BC25_1

### c. Call mutations for all bam

In [ ]:
# Generate commands for call mutations for all splited bam

path = '' # The path containing the split bam file

Sample_BC_Cluster_files = [f for f in os.listdir(path) if f.endswith('.bam')]

print(len(Sample_BC_Cluster_files))
mpileup_command_list = []

for input_file in Sample_BC_Cluster_files:
    input_file_path = os.path.join(path, input_file)
    output_file = input_file.replace(".bam", ".mpileup")
    output_file_path = os.path.join(path, output_file)
    
    mpileup_command = f"samtools mpileup --max-depth 0 --output-BP --reference reference.fa {input_file_path} -o {output_file_path}"
    mpileup_command_list.append(mpileup_command)

with open((path + 'BC_UMI_mpileup_command_list.txt'), "w") as file:
    for mpileup_command in mpileup_command_list:
        file.write("'" + mpileup_command + "'" + '\n')


### d. Generete consensus sequence 


In [ ]:
input_path = '' # The path containing the mpileup file
output_path = '' # The path of output data

reference = SeqIO.read('reference.fa', "fasta")

mpileup_files = [f.split('.')[0] for f in os.listdir(input_path) if f.endswith('.mpileup')]

mpileup_split_mut_total = pd.DataFrame(columns=['pos','ref','alt','ReadsName'])
ConsensueSeq = pd.DataFrame(columns=['ReadsName','seq'])

def replace_with_pipe(match):
    before, n, after = match.groups()
    n = int(n)
    return before + str(n) + after[:n] + '|' + after[n:]

for m_file in mpileup_files:
    mpileup_split = pd.read_table((input_path + m_file + '.mpileup'),\
                                  names = ['chr','pos', 'ref','depth','alt','quality','?','ReadsName']).reset_index()
    mpileup_split = mpileup_split[['pos','ref','depth','alt']].reset_index(drop = True)
    mpileup_split['most_common'] = ''
    mpileup_split['freq'] = ''

    for i in range(len(mpileup_split['pos'])):
        alt = mpileup_split['alt'][i].upper()
        pattern_1 = r'(\D)(\d)([acgtnACGTN]+)'
        alt = re.sub(pattern_1, replace_with_pipe, alt)
        alt = re.sub('\^.','',alt)
        alt = re.sub('\$','',alt)
        pattern = r'([,.][+-]\d+[ACGTNacgtn]+)|([,.<>*])|([ACGTNacgtn])'
        matches = re.findall(pattern, alt)
        alt_list = [item for tup in matches for item in tup if item != '']
        alt_list = [item[1:] if len(item) > 1 else item for item in alt_list]
        element_counts = Counter(alt_list)
        most_common_element = element_counts.most_common(1)[0][0]
        frequency = element_counts[most_common_element] / len(alt_list)
        mpileup_split['most_common'][i] = most_common_element
        mpileup_split['freq'][i] = frequency

    mpileup_split_alt = mpileup_split[(~mpileup_split['most_common'].isin([',','.','*']))&(mpileup_split['freq'] >= 0.6)].reset_index(drop = True)
    
    refstr = list(str(reference.seq).upper())

    for i in range(len(mpileup_split_alt['pos'])):
        pos = mpileup_split_alt['pos'][i] - 1
        ref = mpileup_split_alt['ref'][i]
        alt = mpileup_split_alt['most_common'][i]
        if refstr[pos] == ref:
            if '+' in alt:
                refstr[pos] = refstr[pos] + ''.join(char for char in alt if char.isalpha())
            elif '-' in alt:
                digits = int(''.join(char for char in alt if char.isdigit()))
                for j in range(1,digits + 1):
                    refstr[pos+j] = ''
            else:
                refstr[pos] = alt

    consensus_seq = ''.join(refstr)
    consensus_seq = consensus_seq.upper()
    ConsensueSeq = ConsensueSeq.append({'ReadsName':m_file,'seq':consensus_seq}, ignore_index=True)

    mpileup_split_mut = mpileup_split_alt[['pos','ref','most_common']]
    mpileup_split_mut.columns = ['pos','ref','alt']
    mpileup_split_mut['ReadsName'] = m_file
    
    mpileup_split_mut_total = pd.concat([mpileup_split_mut_total,mpileup_split_mut]).reset_index(drop = True)

BC_name_dict = {'BC01':'RL1','BC03':'RL4','BC04':'RL6','BC05':'RL11','BC07':'RL10','BC08':'RL7','BC09':'A2-L0','BC10':'A2-L1',\
                'BC11':'A2-L2','BC15':'A3-L0','BC18':'B2-L0','BC19':'B2-L1','BC20':'B2-L2','BC27':'B1-L2','BC28':'B1-L3','BC33':'C2-L0',\
                    'BC34':'C2-L1','BC35':'C2-L2','BC36':'C3-L0','BC37':'C3-L1','BC38':'C3-L2','BC39':'C1-L1','BC40':'C1-L2'}

BC_name_df = pd.DataFrame.from_dict(BC_name_dict, orient='index').reset_index()
BC_name_df.columns = ['SampleBC','SampleName']

mpileup_split_mut_total['SampleBC'] = mpileup_split_mut_total['ReadsName'].str.split('_').str.get(0)
mpileup_split_mut_total = pd.merge(mpileup_split_mut_total,BC_name_df, on = 'SampleBC')
mpileup_split_mut_total['SampleName_UMI'] = mpileup_split_mut_total['SampleName'] + '_' + mpileup_split_mut_total['ReadsName'].str.split('_').str.get(-1)

ConsensueSeq['SampleBC'] = ConsensueSeq['ReadsName'].str.split('_').str.get(0)
ConsensueSeq = pd.merge(ConsensueSeq,BC_name_df, on = 'SampleBC')
ConsensueSeq['SampleName_UMI'] = ConsensueSeq['SampleName'] + '_' + ConsensueSeq['ReadsName'].str.split('_').str.get(-1)

mpileup_split_mut_total.to_csv((output_path + 'ConsensusSequence_CallSNP.txt'), sep = '\t')
ConsensueSeq.to_csv((output_path + 'ConsensusSequence.txt'), sep = '\t')

with open((output_path + 'ConsensusSequence.fasta'), 'w') as fasta_consensus:
    for index, row in ConsensueSeq.iterrows():
        reads_id = row['SampleName_UMI']
        seq = row['seq']
        fasta_consensus.write(f'>{reads_id}\n{seq}\n')

## 6. Consensus sequence call SNP

### a. Mutations per allele of each sample

In [3]:
output_path = '' # The path of output data

Mut_total = pd.read_table('ConsensusSequence_CallSNP.txt', usecols = ['pos','ref','alt','SampleName_UMI'])

### # of mutations In each UMI

Sample_UMI_MutCount = Mut_total.value_counts('SampleName_UMI').reset_index()
Sample_UMI_MutCount.columns = ['SampleName_UMI','count']
Sample_UMI_MutCount['sample'] = Sample_UMI_MutCount['SampleName_UMI'].str.split('_').str.get(0)
Sample_UMI_MutCount.loc[Sample_UMI_MutCount['sample'].str.get(0) == 'A', 'Group'] = 'Branch A'
Sample_UMI_MutCount.loc[Sample_UMI_MutCount['sample'].str.get(0) == 'B', 'Group'] = 'Branch B'
Sample_UMI_MutCount.loc[Sample_UMI_MutCount['sample'].str.get(0) == 'C', 'Group'] = 'Branch C'
Sample_UMI_MutCount.loc[Sample_UMI_MutCount['sample'].str.get(0) == 'R', 'Group'] = 'Rosette Leaves'

Sample_UMI_MutCount.to_csv((output_path + 'Sample_MutCount_EachUMI.txt'), sep = '\t')

/tmp/ipykernel_296369/730556737.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Allele_UMI_Count['Redundancy'][i] = '>=5'
/tmp/ipykernel_296369/730556737.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Allele_UMI_Count['Redundancy'][i] = str(Allele_UMI_Count['count'][i])


### b. Frequency of mutations

In [21]:
output_path = '' # The path of output data

Mut_total = pd.read_table('ConsensusSequence_CallSNP.txt', usecols = ['pos','ref','alt','SampleName_UMI','Target'])
Mut_total['sample'] = Mut_total['SampleName_UMI'].str.split('_').str.get(0)
Mut_total['UMI'] = Mut_total['SampleName_UMI'].str.split('_').str.get(1)
Mut_total['mut_info'] = Mut_total['pos'].astype(str) + '_' + Mut_total['ref'] + '_' + Mut_total['alt']

Sample_mut_info_count = Mut_total.value_counts(['sample','mut_info']).reset_index()
Sample_mut_info_count.columns = ['sample','mut_info','mut_info_count']

Sample_count = Mut_total.drop_duplicates('SampleName_UMI').value_counts(['sample']).reset_index()
Sample_count.columns = ['sample','Sample_count']

Sample_mut_info_count = pd.merge(Sample_mut_info_count,Sample_count,on = 'sample')
Sample_mut_info_count['mut_freq'] = Sample_mut_info_count['mut_info_count'] / Sample_mut_info_count['Sample_count']
Sample_mut_info_count = Sample_mut_info_count.sort_values('mut_info').reset_index(drop = True)
Sample_mut_info_count.to_csv((output_path + 'Sample_UMI_MutFreq_Overview.txt'), sep = '\t')

sample_list = list(Sample_mut_info_count['sample'].unique())
sample_list_locals = []

for i in sample_list:
    locals()['MutFreq_' + i] = Sample_mut_info_count[Sample_mut_info_count['sample'] == i][['mut_info','mut_freq']]
    locals()['MutFreq_' + i].columns = ['mut',i]
    sample_list_locals.append(('MutFreq_' + i))

name_1 = sample_list_locals[0]
name_2 = sample_list_locals[1]

MutFreq_merge = pd.merge(locals()[name_1],locals()[name_2], on = 'mut', how = 'outer')

sample_list_locals.remove(name_1)
sample_list_locals.remove(name_2)

for i in sample_list_locals:
    MutFreq_merge = pd.merge(MutFreq_merge,locals()[i], on = 'mut', how = 'outer')

MutFreq_merge = MutFreq_merge.fillna(0)
MutFreq_merge.to_csv((output_path + 'Sample_UMI_MutFreq.txt'), sep = '\t')


,mut,C2-L0,B2-L1,A2-L1,C1-L1,RL7,C1-L2,B2-L0,B1-L2,RL10,...,C3S,A2S,B2S(2),RL4,RL6,C0S,B0S,A3S,C3-L2,B1-L1
0,TargetSite10_830_G_+2CC,0.001727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,TargetSite10_830_G_-267CTAGGGATAACAGGGTAATCACC...,0.001727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,TargetSite10_830_G_A,0.037997,0.004854,0.013043,0.046512,0.007194,0.072727,0.039051,0.009009,0.010811,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,TargetSite10_830_G_C,0.006908,0.000000,0.000000,0.006342,0.000000,0.003636,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,TargetSite10_831_C_G,0.001727,0.004854,0.000000,0.002114,0.000000,0.000000,0.000766,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4924,TargetSite15_1177_T_A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017857
4925,TargetSite16_1294_A_G,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017857
4926,TargetSite1_162_C_A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.053571
4927,TargetSite1_22_G_T,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.053571


# Germline

## 1. Generation of Sanger sequence

In [5]:
output_path = '' # The path of output data
data_path = '' # The path of sanger sequence data

input_files_R = [f.split('_')[0] for f in os.listdir(data_path) if f.endswith('_M13R.seq')]
input_files_F = [f.split('_')[0] for f in os.listdir(data_path) if f.endswith('_M13F.seq')]

difference1 = set(input_files_R) - set(input_files_F)
difference2 = set(input_files_F) - set(input_files_R)
all_differences = difference1.union(difference2)

input_files = input_files_R + input_files_F
input_files = [f for f in input_files if f not in all_differences]
input_files = [(data_path + f) for f in input_files]
input_files = list(set(input_files))

print('# of total files: ' + str(len(input_files)))

def reverse_complement(sequence):
    complement_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    reverse_complement_seq = ''.join(complement_dict[base] for base in reversed(sequence))
    return reverse_complement_seq

def find_common_sequence(seq1, seq2, min_length=50):
    common_sequences = []

    for i in range(len(seq1) - min_length + 1):
        for j in range(min_length, min(len(seq1) - i + 1, len(seq2) + 1)):
            if seq1[i:i+j] in seq2:
                common_sequences.append(seq1[i:i+j])
    return common_sequences

with open((output_path + 'R_F_Overlap.txt'),'w') as R_F_Overlap:
    R_F_Overlap.write('Sample\tConsensus_seq\tR_seq\tF_reverse_seq\tOverlap\n')
    
    with open((output_path + 'Germline_SangerSequence.fasta'),'w') as ConsensusSequence_fa:
        for file in input_files:
            sample = file.split('/')[-1]
            R_file = pd.read_table((file + '_M13R.seq'))
            R_seq = R_file.columns[0]
            F_file = pd.read_table((file + '_M13F.seq'))
            F_seq = F_file.columns[0]
            F_seq_reverse = reverse_complement(F_seq)

            common_seqs = find_common_sequence(R_seq, F_seq_reverse, min_length = 50)
            longest_common_seq = max(common_seqs, key = len, default = 'None')
            print(f"Longest Overlap Sequence: {longest_common_seq}")

            if longest_common_seq != 'None':
                Consensus_seq = R_seq.split(longest_common_seq)[0] + longest_common_seq + F_seq_reverse.split(longest_common_seq)[-1]
                R_F_Overlap.write(sample + '\t' + Consensus_seq + '\t' + R_seq + '\t' + F_seq_reverse + '\t' + longest_common_seq + '\n')
                ConsensusSequence_fa.write('>' + sample + '\n' + Consensus_seq + '\n')

            elif longest_common_seq == 'None':
                Consensus_seq = R_seq
                R_F_Overlap.write(sample + '\t' + Consensus_seq + '\t' + R_seq + '\t' + F_seq_reverse + '\t' + longest_common_seq + '\n')


# of total files: 556
Longest Overlap Sequence: CCTGTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTGTGTCCGCTTGCTAGCCGTATTACCGCTTGTTAGCCGTATTACCGCTTGCTAGCCGTATTACCGCTTGCTAGCCGTATTACCGCTTGCTAGCCGTATTACGCTTTGTGGGTGAATGTATCGATACTCGTGACGACGTATCGGTACTGAGCTCGAATTCACTGGCCTATCGGTACCGATACGCTCGTGTGGGATACGTGCATGCCTGCAGGTCGACTCTAGATGCGATTGATTGCCCGACAACCACTACCTGTATCCCGCTTGCTAGCCGTATTACTGTAGTGTGAACGCAGTATCCATGTCCTTCCTCGTGTATCGGTACCGAGCTCGAATTCACTGGCCCACCGACGTATCTGCCCGACAACCACTACCTGTATCCGCTTGCTAGCCGTATTACATACTACGACGAGTCGTATCTGGGAAGGGGACCGCGTATCGGTACCGAGCTCGAATTCACGGGCCCACCGACGTACCTGCCCGACAACCACTACCTGTATCCGCTTGCTAGCCGTATTACCTTCGACCCTCCGTTGTATCGTTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTGTGCCCGACAACCACTACCTG
Longest Overlap Sequence: TACCTCAGGAAAATTTATTATTATAGGTGATTACCCTGTTATCCCTAGCTGTAGGGATAACAGAGTAATTACAATGTATATCGCGATACCGGCTGCGGCTCATACAGTTGCCTTTGATAGCGCTT

## 2. Sanger sequence call SNP and generate consensus offspring sequence

### a. Sanger sequence call SNP

In [7]:
# Sanger consensus sequence call SNP
# nohup minimap2 -ax map-ont -t 50 reference.fa Germline_SangerSequence.fa -o Germline_SangerSequence.fa.bam &
# nohup samtools sort Germline_SangerSequence.fa.bam -o Germline_SangerSequence.fa.sorted.bam &
# nohup samtools mpileup --max-depth 0 --output-BP --output-QNAME --reference reference.fa Germline_SangerSequence.fa.sorted.bam -o Germline_SangerSequence.mpileup &

mpileup_file = pd.read_table('Germline_SangerSequence.mpileup', names = ['pos', 'ref','depth','alt','quality','?','ReadsName'])
mpileup_file = mpileup_file[mpileup_file['depth'] != '0'].reset_index().drop('?', axis = 1)
mpileup_file.columns = ['chr','pos','ref','depth','alt','quality','ReadsName']

mpileup_file_HaveMut = mpileup_file[mpileup_file['alt'].str.contains('[a-zA-Z]')].reset_index(drop = True)

def replace_with_pipe(match):
    before, n, after = match.groups()
    n = int(n)
    return before + str(n) + after[:n] + '|' + after[n:]

Mut_reads_unique = pd.DataFrame(columns=['chr', 'pos', 'ref','alt','ReadsName'])

for i in range(len(mpileup_file_HaveMut['chr'])):
    print(i)
    ref_chr = mpileup_file_HaveMut['chr'][i]
    ref = mpileup_file_HaveMut['ref'][i].upper()
    pos = mpileup_file_HaveMut['pos'][i]
    alt = mpileup_file_HaveMut['alt'][i].upper()
    
    pattern_1 = r'(\D)(\d+)([acgtnACGTN]+)'
    alt = re.sub(pattern_1, replace_with_pipe, alt)
    
    alt = re.sub('\^.','',alt)
    alt = re.sub('\$','',alt)
    pattern = r'([,.][+-]\d+[ACGTNacgtn]+)|([,.<>*])|([ACGTNacgtn])'
    matches = re.findall(pattern, alt)
    alt_list = [item for tup in matches for item in tup if item != '']
    ReadsName_list = mpileup_file_HaveMut['ReadsName'][i].split(',')
    for j in range(len(ReadsName_list)):
        Mut_reads_unique = Mut_reads_unique.append({'chr':ref_chr,'pos':pos,'ref': ref,\
            'alt': alt_list[j],'ReadsName':ReadsName_list[j]}, ignore_index=True)

Mut_reads_unique = Mut_reads_unique[~Mut_reads_unique['alt'].isin([',','.','*','>','<'])]
Mut_reads_unique['serial_number'] = Mut_reads_unique['ReadsName'].str.split('-').str.get(0)
Mut_reads_unique['clone'] = Mut_reads_unique['ReadsName'].str.split('-').str.get(-1)

serial_number_Germline = pd.read_table('serial_number.txt')
serial_number_Germline['serial_number'] = serial_number_Germline['serial_number'].apply(str)

Mut_reads_unique = pd.merge(Mut_reads_unique,serial_number_Germline,on = 'serial_number')
Mut_reads_unique['alt'] = Mut_reads_unique['alt'].str.replace('[,.]', '', regex=True)
Mut_reads_unique.to_csv('Mutation.txt', sep = '\t')
Mut_reads_unique

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,chr,pos,ref,alt,ReadsName,serial_number,clone,Germline
0,ref,8,C,-1A,7-14,7,14,3-1-2-4
1,ref,21,A,T,7-5,7,5,3-1-2-4
2,ref,21,A,T,7-3,7,3,3-1-2-4
3,ref,41,C,-1A,7-14,7,14,3-1-2-4
4,ref,113,C,T,7-12,7,12,3-1-2-4
...,...,...,...,...,...,...,...,...
15804,ref,1234,G,T,38-9,38,9,2-1-3-1
15805,ref,1236,C,T,38-9,38,9,2-1-3-1
15806,ref,1237,T,C,38-9,38,9,2-1-3-1
15807,ref,1002,G,T,2-1,2,1,2-1-3-1


### b. Identify copies

In [ ]:
consensus_seq = pd.read_table('R_F_Overlap.txt')
consensus_seq = consensus_seq[consensus_seq['Overlap'] != 'None'][['Sample']]
consensus_seq['serial_number'] = consensus_seq['Sample'].str.split('-').str.get(0)
consensus_seq['clone'] = consensus_seq['Sample'].str.split('-').str.get(1)

serial_number = pd.read_table('serial_number.txt')
serial_number['serial_number'] = serial_number['serial_number'].apply(str)
consensus_seq = pd.merge(consensus_seq,serial_number,on = 'serial_number')


Mut_reads_unique = pd.read_table('Mutation.txt',\
    usecols = ['pos', 'ref', 'alt', 'ReadsName', 'serial_number', 'clone', 'Germline'])
Mut_reads_unique['mut_info'] = Mut_reads_unique['pos'].astype(str) + '_' + Mut_reads_unique['ref'] + '_' + Mut_reads_unique['alt']

Germline_list = Mut_reads_unique['Germline'].unique()

for Germline in Germline_list:
    consensus_seq_Germline = consensus_seq[consensus_seq['Germline'] == Germline]
    All_clone_list = consensus_seq_Germline['clone'].astype(int).unique()
    Mut_reads_unique_subset = Mut_reads_unique[Mut_reads_unique['Germline'] == Germline].reset_index(drop = True)
    clone_list = Mut_reads_unique_subset['clone'].unique()
    mut_info_list = Mut_reads_unique_subset['mut_info'].unique()
    mut_clone_df = pd.DataFrame(index = mut_info_list, columns = clone_list)
    for clone in All_clone_list:
        if clone in clone_list:
            Mut_reads_unique_clone = Mut_reads_unique_subset[Mut_reads_unique_subset['clone'] == clone].reset_index(drop = True)
            for mut in mut_info_list:
                if mut in Mut_reads_unique_clone['mut_info'].unique():
                    mut_clone_df[clone][mut] = 1
                else:
                    mut_clone_df[clone][mut] = 0
        else:
            mut_clone_df[clone] = 0

    mut_clone_df.to_csv((Germline + '.txt'), sep = '\t')

In [35]:
output_path = '' # The path of output data

Germline_copy_dict = {'1-2-1-1':['1','3','13','9','8','12','10','17','14','15','16'],'1-2-3-1':['1','13','17','23','16','22','21','19','12','6','24','20','18'],
                      '1-2-3-2':['3','9','5','7','14','11','13','1','8','12','10','2','4'],'1-3-1-2':['8','9','11','13','2','14','7','12','5','10'],
                      '1-3-2-1':['4','2','14','19','18','20','17','23','24','25','16','22'],'1-3-2-2':['1','9','13','3','7','10','14'],'1-3-4-1':['14','2','4','5','13','6','1','12','3'],[],
                      '1-3-5-1':['2','6','1','3','4'],'1-3-5-1b':['1','12','3','11','10','6'],
                      '2-1-4-2':['4','8','3','10','11','12','13','6','5','1','14','2'],'2-1-9-2':['3','10','11','2','14','6','5'],
                      '2-2-1-2':['14','3','11','6','4','10','1','9','12','7'],'2-2-6-1':['9','2','3','11','12','7','10','5','14','13','8','1'],
                      '2-2-8-4':['8','4','14','2','13','3','9','11','6','1','12'],'3-1-2-1':['1','5','4'],'3-1-2-4':['14','13','3','8','4','5','7','1','2'],
                      '3-1-2-6':['6','7','5','12'],'3-1-4-1':['6','4','8','3','10','11','12','9','7','1','5','2'],
                      '3-2-4-1':['8','9','10','13','11','12','7','1','2','5','4','14','6'],'3-2-5-2':['2','7','4','12','3','14','6','8','5','1','10','11','13'],
                      '3-3-3-2':['14','2','8','6','11','9','4','10','1','12','7','13'],'3-3-3-3':['4','1','15','11','9','3','13','10','8','2','5','6','12','16','14'],'3-3-3-4':['1','5','6','14','16','8','11','2','3','4','17','15','9','12','10','13']}

Mut_Fraction_cutoff = 0.8

for Germline in Germline_copy_dict:
    mut_clone_df = pd.read_table(Germline + '.txt')
    mut_clone_df = mut_clone_df.set_index('Unnamed: 0')
    mut_clone_df.index.name = None
    
    mut_copy1_df = mut_clone_df[Germline_copy_dict[Germline]]
    if mut_copy1_df.shape[1] >= 2:
        mut_copy1_df['Mut_Fraction'] = mut_copy1_df.sum(axis=1) / mut_copy1_df.shape[1]
    else:
        mut_copy1_df['Mut_Fraction'] = 0
    mut_copy1_df['Germline_copy'] = Germline# + '-1'
    mut_copy1_df['clone_count'] = len(Germline_copy_dict[Germline][0])
    mut_copy1_df = mut_copy1_df.reset_index()['index','Mut_Fraction','Germline_copy','clone_count']

    mut_copy_df = mut_copy1_df.copy()

    mut_copy_df = mut_copy_df[mut_copy_df['Mut_Fraction'] >= Mut_Fraction_cutoff].reset_index(drop=True)
    mut_copy_df.columns = ['mut_info','Mut_Fraction','Germline_copy','clone_count']

    mut_copy_df['pos'] = mut_copy_df['mut_info'].str.split('_').str.get(0)
    mut_copy_df['ref'] = mut_copy_df['mut_info'].str.split('_').str.get(1)
    mut_copy_df['alt'] = mut_copy_df['mut_info'].str.split('_').str.get(2)

    mut_copy_df.to_csv((output_path + Germline + '_Cutoff' + str(Mut_Fraction_cutoff) + '.txt'), sep = '\t', index = False)

# Merge the mutation files
# awk 'FNR == 1 && NR != 1 {next} {print}' $(ls *.txt) > Mut_Identified.txt

/tmp/ipykernel_125962/3384543711.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mut_copy1_df['Mut_Fraction'] = mut_copy1_df.sum(axis=1) / mut_copy1_df.shape[1]
/tmp/ipykernel_125962/3384543711.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mut_copy1_df['Germline_copy'] = Germline# + '-1'
/tmp/ipykernel_125962/3384543711.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

## 3. Calculation of mutation frequency

In [3]:
Germline_mutation = pd.read_table('Mut_Identified.txt')
Germline_mutation['mut_info'] = Germline_mutation['pos'].astype(str) + '_' + Germline_mutation['ref'] + '_' + Germline_mutation['alt']
Germline_mutation['Branch'] = Germline_mutation['Germline_copy'].str.get(0).astype(str) + '_Branch_Germline'
replace_dict = {'1': 'A', '2': 'B', '3': 'C'}

def replace_numbers(match):
    return replace_dict[match.group(0)]
Germline_mutation['Branch'] = Germline_mutation['Branch'].apply(lambda x: re.sub(r'\d', replace_numbers, x))

Branch_list = Germline_mutation['Branch'].unique()
mut_list = Germline_mutation['mut_info'].unique()

Gemline_Branch_mutation = pd.DataFrame(index = mut_list, columns = Branch_list)

for Branch in Branch_list:
    Germline_mutation_subset = Germline_mutation[Germline_mutation['Branch'] == Branch]
    Germline_mutation_subset_mut = Germline_mutation_subset['mut_info'].unique()
    print('# mutation in ' + Branch + ': ' + str(len(Germline_mutation_subset_mut)))
    for mut in mut_list:
        if mut in Germline_mutation_subset_mut:
            freq = len(Germline_mutation_subset[Germline_mutation_subset['mut_info'] == mut]['Germline_copy'].unique()) / \
                len(Germline_mutation_subset['Germline_copy'].unique())
            Gemline_Branch_mutation[Branch][mut] = freq

Gemline_Branch_mutation = Gemline_Branch_mutation.fillna(0)
Gemline_Branch_mutation.to_csv('Germline_mutation_Cutoff.txt', sep = '\t')
Gemline_Branch_mutation = Gemline_Branch_mutation.reset_index()
Gemline_Branch_mutation = Gemline_Branch_mutation.rename(columns={'index': 'mut'})

MutFreq = pd.read_table('Sample_UMI_MutFreq.txt')
MutFreq['mut'] = MutFreq['mut'].apply(lambda x: '_'.join(x.split('_')[1:]))

MutFreq = pd.merge(Gemline_Branch_mutation, MutFreq, on = 'mut', how = 'outer').\
    drop(['Unnamed: 0'], axis = 1).fillna(0)
MutFreq.to_csv('Total_Mutation_Freq.txt', sep = '\t')

# mutation in A_Branch_Germline: 50
# mutation in B_Branch_Germline: 59
# mutation in C_Branch_Germline: 91


/tmp/ipykernel_9588/2616666285.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Gemline_Branch_mutation_info['sample'] = 'Offsprings'
/tmp/ipykernel_9588/2616666285.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Gemline_Branch_mutation_info['Group_big'] = 'Offsprings'


,Parent,Parent_Mut,PassToOffspring,Fraction
0,Branch A,386,1,0.002591
1,Branch B,1038,1,0.000963
2,Branch C,689,1,0.001451
3,Rosette Leaves,494,0,0.0
4,Branch AB,94,0,0.0
5,Branch AC,68,1,0.014706
6,Branch BC,185,0,0.0
7,AR,58,1,0.017241
8,BR,132,0,0.0
9,CR,105,0,0.0


## 4. Tree

### a. Generation of germline consensus sequence

In [17]:
Germline_mutation = pd.read_table('Mut_Identified.txt')

reference = SeqIO.read('reference.fa', "fasta")

Germline_list = Germline_mutation['Germline_copy'].unique()
Germline_ConsensueSeq = pd.DataFrame(columns=['Germline','seq'])

for Germline in Germline_list:
    Germline_mutation_subset = Germline_mutation[Germline_mutation['Germline_copy'] == Germline].reset_index(drop = True)
    refstr = list(str(reference.seq).upper())
    for i in range(len(Germline_mutation_subset['pos'])):
        pos = Germline_mutation_subset['pos'][i] - 1
        ref = Germline_mutation_subset['ref'][i]
        alt = Germline_mutation_subset['alt'][i]
        if refstr[pos] == ref:
            #print('Yes')
            if '+' in alt:
                refstr[pos] = refstr[pos] + ''.join(char for char in alt if char.isalpha())
            elif '-' in alt:
                digits = int(''.join(char for char in alt if char.isdigit()))
                for j in range(1,digits + 1):
                    refstr[pos+j] = ''
            else:
                refstr[pos] = alt
        
    consensus_seq = ''.join(refstr)
    
    consensus_seq = consensus_seq.upper()
    Germline_ConsensueSeq = Germline_ConsensueSeq.append({'Germline':Germline,'seq':consensus_seq}, ignore_index=True)

Germline_ConsensueSeq.to_csv(('Germline_ConsensusSeq.txt'), sep = '\t')

with open('Germline_ConsensusSeq.fasta', 'w') as fasta_consensus:
    for index, row in Germline_ConsensueSeq.iterrows():
        Germline = row['Germline']
        seq = row['seq']
        fasta_consensus.write(f'>{Germline}\n{seq}\n')

fasta_file1 = list(SeqIO.parse('Germline_ConsensusSeq.fasta', "fasta"))
fasta_file2 = list(SeqIO.parse('ConsensusSequence.fasta', "fasta")) # UMIC-seq consensus sequence
fasta_file_combined = fasta_file1 + fasta_file2


WT_seq = str(reference.seq)

BC_name_list = ['RL1','RL4','RL6','RL11','RL10','RL7','A2-L0','A2-L1','A2-L2','A3-L0','B2-L0','B2-L1','B2-L2',\
                'B1-L2','B1-L3','C2-L0','C2-L1','C2-L2','C3-L0','C3-L1','C3-L2','C1-L1','C1-L2']
BC_name_list = BC_name_list + list(Germline_list)

with open('ConsensusSeq.fasta', "w") as output_handle:
    for record in fasta_file_combined:
        if str(record.seq).upper() != WT_seq.upper() and record.id.split('_')[0] in BC_name_list:
            output_handle.write('>' + record.id.replace('-','_') + '\n' + str(record.seq).upper() + '\n')